# Multi-Sensor Data Fusion for Remote Sensing

This notebook demonstrates advanced techniques for combining Sentinel-2 and Landsat imagery to enhance spatial and spectral resolution using data fusion methods, such as pansharpening and feature stacking, with `rasterio`, `geopandas`, and `numpy` in Python. Multi-sensor fusion is useful for improving analysis accuracy in applications like land cover classification, change detection, or vegetation monitoring.

## Prerequisites
- Install required libraries: `rasterio`, `geopandas`, `numpy`, `matplotlib`, `pyproj`, `scikit-image` (listed in `requirements.txt`).
- Preprocessed Sentinel-2 and Landsat GeoTIFF files (e.g., `sentinel_rgb.tif` and `landsat_rgb.tif` from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A GeoJSON or shapefile defining the area of interest (AOI) (e.g., `aoi.geojson`).
- Replace file paths with your own data.

## Learning Objectives
- Align Sentinel-2 and Landsat rasters to a common CRS and resolution.
- Perform pansharpening to enhance Sentinel-2 resolution using Landsat's panchromatic band.
- Create a fused feature stack combining multi-spectral bands from both sensors.
- Visualize and save fused datasets for downstream analysis.

In [ ]:
# Import required libraries
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.warp import reproject, Resampling
from rasterio.merge import merge
from pyproj import CRS
from skimage.transform import resize
import os

## Step 1: Load Sentinel-2 and Landsat Data

Load preprocessed Sentinel-2 and Landsat GeoTIFFs and the AOI vector data, ensuring compatibility in CRS.

In [ ]:
# Define file paths
sentinel_path = 'remote_sensing_data/sentinel_rgb.tif'  # Replace with your Sentinel-2 GeoTIFF
landsat_path = 'remote_sensing_data/landsat_rgb.tif'   # Replace with your Landsat GeoTIFF
aoi_path = 'aoi.geojson'                              # Replace with your AOI file

# Load Sentinel-2 data
with rasterio.open(sentinel_path) as src_s2:
    sentinel_data = src_s2.read(masked=True)  # Shape: (bands, height, width)
    sentinel_profile = src_s2.profile
    sentinel_crs = src_s2.crs

# Load Landsat data
with rasterio.open(landsat_path) as src_l8:
    landsat_data = src_l8.read(masked=True)
    landsat_profile = src_l8.profile
    landsat_crs = src_l8.crs

# Load AOI
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != sentinel_crs:
    aoi_gdf = aoi_gdf.to_crs(sentinel_crs)

# Print basic information
print(f'Sentinel-2 shape: {sentinel_data.shape}, CRS: {sentinel_crs}')
print(f'Landsat shape: {landsat_data.shape}, CRS: {landsat_crs}')
print(f'AOI CRS: {aoi_gdf.crs}')

## Step 2: Align Rasters to Common CRS and Resolution

Reproject and resample Landsat data to match Sentinel-2's CRS and resolution (10m).

In [ ]:
# Define target CRS and resolution (use Sentinel-2 as reference)
target_crs = sentinel_crs
target_transform = sentinel_profile['transform']
target_width, target_height = sentinel_profile['width'], sentinel_profile['height']

# Reproject Landsat to match Sentinel-2
aligned_landsat_data = np.empty((landsat_data.shape[0], target_height, target_width), dtype=np.float32)
with rasterio.open(landsat_path) as src:
    for i in range(landsat_data.shape[0]):
        reproject(
            source=landsat_data[i],
            destination=aligned_landsat_data[i],
            src_transform=landsat_profile['transform'],
            src_crs=landsat_crs,
            dst_transform=target_transform,
            dst_crs=target_crs,
            resampling=Resampling.bilinear
        )

# Update Landsat profile
aligned_landsat_profile = sentinel_profile.copy()
aligned_landsat_profile.update({'count': landsat_data.shape[0]})

# Visualize aligned Landsat RGB
aligned_landsat_rgb = aligned_landsat_data[:3].transpose(1, 2, 0)
aligned_landsat_rgb = aligned_landsat_rgb / np.nanpercentile(aligned_landsat_rgb, 98) if np.nanpercentile(aligned_landsat_rgb, 98) > 0 else aligned_landsat_rgb
aligned_landsat_rgb = np.clip(aligned_landsat_rgb, 0, 1)

plt.figure(figsize=(8, 8))
plt.imshow(aligned_landsat_rgb)
plt.title('Aligned Landsat RGB Composite')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 3: Pansharpening with Landsat Panchromatic Band

Use Landsat's panchromatic band (15m) to enhance the resolution of Sentinel-2 RGB bands (10m).

In [ ]:
# Load Landsat panchromatic band (Band 8, 15m resolution)
landsat_pan_path = 'remote_sensing_data/landsat_pan.tif'  # Replace with your Landsat panchromatic GeoTIFF
with rasterio.open(landsat_pan_path) as src_pan:
    pan_data = src_pan.read(1, masked=True)  # Shape: (height, width)
    pan_profile = src_pan.profile
    pan_crs = src_pan.crs

# Reproject panchromatic band to Sentinel-2 resolution and CRS
aligned_pan_data = np.empty((target_height, target_width), dtype=np.float32)
with rasterio.open(landsat_pan_path) as src:
    reproject(
        source=pan_data,
        destination=aligned_pan_data,
        src_transform=pan_profile['transform'],
        src_crs=pan_crs,
        dst_transform=target_transform,
        dst_crs=target_crs,
        resampling=Resampling.bilinear
    )

# Simple Brovey pansharpening
pansharpened_data = np.zeros_like(sentinel_data, dtype=np.float32)
for i in range(3):  # Process RGB bands
    pansharpened_data[i] = sentinel_data[i] * (aligned_pan_data / np.nanmean(aligned_landsat_data[:3], axis=0))

# Visualize pansharpened RGB
pansharpened_rgb = pansharpened_data[:3].transpose(1, 2, 0)
pansharpened_rgb = pansharpened_rgb / np.nanpercentile(pansharpened_rgb, 98) if np.nanpercentile(pansharpened_rgb, 98) > 0 else pansharpened_rgb
pansharpened_rgb = np.clip(pansharpened_rgb, 0, 1)

plt.figure(figsize=(8, 8))
plt.imshow(pansharpened_rgb)
plt.title('Pansharpened Sentinel-2 RGB Composite')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Save pansharpened raster
pansharpened_output_path = 'remote_sensing_data/pansharpened_raster.tif'
with rasterio.open(pansharpened_output_path, 'w', **sentinel_profile) as dst:
    dst.write(pansharpened_data)

print(f'Pansharpened raster saved to: {pansharpened_output_path}')

## Step 4: Create Fused Feature Stack

Combine Sentinel-2 and Landsat bands into a single feature stack for enhanced analysis.

In [ ]:
# Combine Sentinel-2 and aligned Landsat bands
fused_data = np.concatenate([sentinel_data, aligned_landsat_data], axis=0)

# Update profile for fused data
fused_profile = sentinel_profile.copy()
fused_profile.update({'count': fused_data.shape[0]})

# Visualize first three bands of fused stack
fused_rgb = fused_data[:3].transpose(1, 2, 0)
fused_rgb = fused_rgb / np.nanpercentile(fused_rgb, 98) if np.nanpercentile(fused_rgb, 98) > 0 else fused_rgb
fused_rgb = np.clip(fused_rgb, 0, 1)

plt.figure(figsize=(8, 8))
plt.imshow(fused_rgb)
plt.title('Fused Feature Stack (First 3 Bands)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Save fused feature stack
fused_output_path = 'remote_sensing_data/fused_feature_stack.tif'
with rasterio.open(fused_output_path, 'w', **fused_profile) as dst:
    dst.write(fused_data)

print(f'Fused feature stack saved to: {fused_output_path}')

## Step 5: Validate Fusion with AOI Overlay

Overlay the AOI on the fused data to verify alignment and coverage.

In [ ]:
# Visualize fused data with AOI overlay
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(fused_rgb)
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
plt.title('Fused Feature Stack with AOI Overlay')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `sentinel_rgb.tif`, `landsat_rgb.tif`, and `landsat_pan.tif` with your own GeoTIFF files (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Experiment with other pansharpening methods (e.g., PCA-based or Gram-Schmidt) for improved results.
- Use the fused feature stack in downstream tasks like classification (see `12_classification_rf_svm.ipynb`) or segmentation (see `15_unet_segmentation.ipynb`).
- Explore advanced fusion techniques like deep learning-based methods for super-resolution.

## Notes
- Ensure Sentinel-2 and Landsat rasters are preprocessed (e.g., cropped and aligned) as shown in `24_advanced_preprocessing.ipynb`.
- Pansharpening assumes the panchromatic band is available; if not, skip to feature stacking.
- Handle no-data values carefully to avoid artifacts in fused data.
- See `docs/installation.md` for troubleshooting library installation.